In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pickle
import time
import requests
import os
from scraping import *
import pandas as pd
import regex as re
import numpy as np
import random

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:

chromedriver = "E:/chromedriver_win32/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)


In [151]:
url = "https://www.tripadvisor.in/Attraction_Review-g61000-d139187-Reviews-or5-Glacier_Point-Yosemite_National_Park_California.html#REVIEWS"
driver.get(url)
time.sleep(10)
button = driver.find_element_by_class_name("_3aUwQbpX")
button.click()
ta_reviews_page_soup = BeautifulSoup(driver.page_source,'html5lib')


In [4]:
url = "https://www.tripadvisor.in/Attraction_Review-g61000-d139187-Reviews-or{}-Glacier_Point-Yosemite_National_Park_California.html#REVIEWS"

col_names = ['attraction_name', 
 'attraction_id',
 'user_name',
 'user_profile_link',
 'review_date',
 'helpful_votes',
 'rating',
 'review_link',
 'review_text',
 'review_title',
 'experience_date']

df_reviews = pd.DataFrame(columns=col_names)

# point_name = soup.find('title').text.split(' - ')[0]
# point_id = '-'.join(url.split('-')[1:3])

In [29]:
df_reviews['review_text'][500]

&quot;I&#39;d seen the pictures before we visited from England and assumed they must have been touched up. But this is breathtaking. Came twice as the first time it was a bit hazy due to forest fires.  Sunset here is hard to describe. If you ever get the chance go!&quot;

In [11]:
attraction_url_list = ['https://www.tripadvisor.in/Attraction_Review-g61000-d489919-Reviews-or{}-Yosemite_Valley-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d592681-Reviews-or{}-Mariposa_Grove_of_Giant_Sequoias-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d139183-or{}-Reviews-Half_Dome-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d593212-or{}-Reviews-Tunnel_View-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d146103-or{}-Reviews-Tioga_Pass-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d139182-or{}-Reviews-El_Capitan-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d483482-or{}-Reviews-Mist_Trail-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d483476-or{}-Reviews-Yosemite_Falls-Yosemite_National_Park_California.html',
'https://www.tripadvisor.in/Attraction_Review-g61000-d483481-or{}-Reviews-Vernal_Fall-Yosemite_National_Park_California.html']

attraction_name_list = [re.sub('_', ' ', name.split('-')[-2]) for name in attraction_url_list]

attraction_numbers = [2000, 1861, 1472, 2000, 1159, 1362, 1020, 2000, 1383]

attraction_id_list = ['-'.join(url.split('-')[1:3]) for url in attraction_url_list]

In [30]:
df_reviews_total = pd.read_csv('attraction_point_reviews.csv')
# df_reviews_total.tail()

In [31]:

for i in range(4,9):

    url = attraction_url_list[i]
    point_name = attraction_name_list[i]
    point_id = attraction_id_list[i]
    n = attraction_numbers[i]
    df_reviews = pd.DataFrame(columns=col_names) 

    for j in range(0, n+1, 5):

        try:
            response = requests.get(url.format(j))
            if response.status_code == 200:

                soup = BeautifulSoup(response.text, 'html5lib')
                reviews_list = ta_attraction_reviews_parser(soup)
                
                if reviews_list:
                    df_reviews = df_reviews.append(reviews_list, ignore_index = True)

                    df_reviews['attraction_name'] = point_name
                    df_reviews['attraction_id'] = point_id

                    df_reviews.to_csv(f"attraction_point_reviews_{point_name}.csv" , index = False)

                else:
                    print(f"Reviews List came up empty on url {url}")
                    # input()

            else:
                print(f"status code returned is {response.status_code} for page {url}")
                # input()
        
        except Exception as err_message:
            print(f"For page {url}, received following error message: {err_message}")

        # print(df_reviews.tail(2))
        # time.sleep(0.5*random.random())

    df_reviews.to_csv(f"attraction_point_reviews_{point_name}.csv" , index = False)
    df_reviews.tail()

    df_reviews_total = pd.concat([df_reviews_total, df_reviews])
    df_reviews_total.to_csv("attraction_point_reviews.csv", index = False)

df_reviews_total.tail()
             


    


KeyboardInterrupt: 

In [17]:
i

3